# GGR376 LAB 5.2: Visualizing LiDAR Data using Python (4 marks)

In this part of the lab, we will be using the **laspy** library to read and store the LiDAR LAS data from Lab 2. The LiDAR data are stored as 3-dimensional XYZ **arrays**, where data for each point are organised into three columns. X and Y represent the horizontal position (east-west and north-south positions), while Z represents the height (elevation) of the point above the Earth's surface.

Not all Python libraries are pre-installed and ready for import on the Jupyter hub. To use laspy, you therefore need to install it each time you open a new notebook in the hub - pip is a package management tool that easily allows you to do this.

We will also use the **numpy** library which is particularly helpful when working with array data structures. Lastly, we will continue working with the pyplot module in the **matplotlib** library to visualize the point cloud data. More information about each library can be accessed via the following links:

- [laspy](https://laspy.readthedocs.io/en/latest/index.html)
- [numpy](https://numpy.org/devdocs/user/absolute_beginners.html)
- [matplotlib](https://matplotlib.org/stable/api/pyplot_summary.html)

Again, as you work through the notebook, look out for **Your turn** sections in which you are required to write your own code. As a reminder, all code must be clearly commented to demonstrate your understanding.

In [ ]:
# Install laspy
!pip install laspy

In [ ]:
# Import required libraries (note numpy and matplot are already installed on the hub)
import laspy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read in the LAS data using the laspy read() function
las = laspy.read('data/utor_original.las')

# View details from the xml metdata (e.g., point format and the number of points within the dataset)
las

In [ ]:
#View the attribute value names associated with the point cloud data
list(las.point_format.dimension_names)

### Explore attribute values as arrays
An array is a single variable which stores multiple values **(see Python handbook for a reminder)**. 

You can call any of the attribute field names and view the output as an array by typing `las.fieldname`. 

You can also view [summary statistics](https://numpy.org/doc/stable/reference/routines.statistics.html) by appending a numpy function to the fieldname e.g., `print("Mean:", las.fieldname.mean())`

### Your turn (1 mark):
In the cell below, explore some of the values by replacing `fieldname` with a name of one of the fields listed in the cell above. Do you notice anything unusual about the X, Y and Z values? 

To view the 3-dimensional array, try calling las.xyz

*Tip: If you'd like to add a new cell into a notebook, hold ESC and B keys or use the **+** symbol from the menu*

In [ ]:
# Explore attribute values as arrays
# Insert your code here and run the cell...



### Explore the available classification codes
The classification field contains the numbers that represent different classes. Rather than view all of the potential values as an array, we can convert the array to a Python set which stores unique values. You can visit https://desktop.arcgis.com/en/arcmap/latest/manage-data/las-dataset/lidar-point-classification.htm as a reminder for what classes the code values refer to.

*Tip: If you click on the name of a function in a code cell and then hold SHIFT and TAB keys on your keyboard, you can access help documentation. Then expand the window for more details*

In [ ]:
# View the unique classification codes
set(list(las.classification)) # Convert classificatin attribute into Python list then convert list into set of unique values

Identifying the classification codes may be useful if you want to export points associated with a particular object on the Earth's surface, such as ground, buildings or vegetation. The following lines of code demonstrate how to do this for points classified as '2-ground'. 

Note that the new las file could be exported to your data folder by calling `ground.write('data/filename.las')`

In [ ]:
# Create a new LAS file for storing points classified as ground with the same format as our original file 'las'
ground = laspy.create(point_format=las.header.point_format, file_version=las.header.version)

# Assign points classified as 2 (ground) from original las file to points attribute of new ground las file
ground.points = las.points[las.classification == 2]

### 3D point cloud visualization
Again, Python provides us with an excellent tool for EDA. We already have an idea of the file's different attribute values but it is also helpful to explore the visual output of the data.

Let's start by creating a basic 2D scatter plot using the matplotlib [.scatter()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) function. We can set the colour of the points to our Z value here.

In [ ]:
# Create a scatter plot with colour changing based on Z values
plt.scatter(las.X, 
            las.Y, # XY data positions
            c=las.Z, # marker colours
            cmap='viridis') # colour scheme

plt.colorbar(label='Heights above sea level')  # Add colour bar to show Z values

plt.xlabel('X')
plt.ylabel('Y')
plt.title('2D scatter plot of LiDAR points with heights shown')

plt.show()

A 2D plot is helpful for a first glance of the data, but we can also view the data in 3D by adding a 3D subplot and including Z values as a parameter in our plot.

You may wish to refer to the matplotlib [.scatter()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) documentation or [W3Schools tutorial](https://www.w3schools.com/python/matplotlib_scatter.asp) to understand how to update the colour scheme, change the marker symbol etc.

In [ ]:
# Create a figure object with a specified size
fig = plt.figure(figsize=(10, 8))

# Add a subplot to the figure with a 3D projection
ax = fig.add_subplot(111, projection='3d')

# Scatter plot points from 'las' data on the 3D axes
# las.X, las.Y, and las.Z are assumed to be arrays or lists of x, y, and z coordinates
ax.scatter(las.X, 
           las.Y,
           las.Z, # XYZ data positions
           c=las.Z, # marker colours
           cmap='viridis') # colour scheme

# Set labels for each axis
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set the title of the plot
plt.title('3D scatter plot of LiDAR points')

# Display  plot
plt.show()

Notice that the output was a little slow as we are plotting over 3.5 million points! We can also see that our heights (and eastings and northings) appear to be in cm.

For EDA, it may be helpful just to view a subset of point data for speed. In the following code we convert the Z values to metres above sea level and create a random subest of 1% of the data

In [ ]:
# Create new variable by completing the code below and converting Z values from cm to metres
las.Zm = las.Z / 100
las.Zm

In [ ]:
# Get total number of points in 'las' and store as a variable
numpts = len(las.X)

# Return new array of false values with the same length as the number of points
mask = np.full(numpts, False)

# Calculate the number of points to set as True (1% of numpts)
mask[:int(numpts*0.01)] = True

# Shuffle mask array to randomly distribute True values within it
np.random.shuffle(mask)

#Create new variables containing XYZ values from original data respectively. Use only data points from las where corresponding mask values are true
xmask = las.X[mask]
ymask = las.Y[mask]
zmask = las.Zm[mask] # Use updated Z values here. You may also think about how to update XY values to m


### Your turn (3 marks):
Using the code above for reference, create the following visualization:
- 3D scatter plot of random subset of LiDAR data


Trial different options here as we did in lab 5.1, such as adding a colourbar, incorporating a title, removing axes.

In [ ]:
# 3D Scatter plot of subset of LiDAR data
# Insert your code here and run the cell...



The static 3D output is a little limited compared with the Scene layers we have been working with in ArcGIS Pro. Although not available to us through the UofT Jupyter hub, you may like to take a look at the following links to explore the potential capabilities of creating 3D interactive visualizations in just a couple of lines of code.

* geemap library: https://geemap.org/notebooks/101_lidar
* open3d library: http://www.open3d.org/docs/0.9.0/tutorial/Basic/pointcloud.html

Once you have completed the lab, export your notebook by selection File > Print Preview > Save as PDF 

Upload your PDF to Part 2 of the lab.